## **Federated Learning for attack detection: 7 nodes sharing weights**

IDs from this file = **id11xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Transform to One Hot Encoding the Categories - normal, attack
    data_labels.insert(0, 'normal', data['attack_cat'].replace('normal', 1).replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'attack', data['attack_cat'].replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 1).replace('normal', 0))

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 7A

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part5.csv')
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part6.csv')
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part7.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_748/1809786618.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [6]:
# Define 
node_datasets = [training1, training2, training3, training4, training5, training6, training7]
num_nodes = 7
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1100.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [11]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5: 
            x, y = x6, y6
        else: 
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1100.hdf5')

Epoch 1/15
23/23 [==============================] - 37s 2s/step - loss: 0.4570 - accuracy: 0.8573 - val_loss: 1.0714 - val_accuracy: 0.6409
Epoch 2/15
23/23 [==============================] - 29s 1s/step - loss: 0.1058 - accuracy: 0.9863 - val_loss: 2.1137 - val_accuracy: 0.6413
Epoch 3/15
23/23 [==============================] - 33s 1s/step - loss: 0.0468 - accuracy: 0.9920 - val_loss: 1.0029 - val_accuracy: 0.6422
Epoch 4/15
23/23 [==============================] - 36s 2s/step - loss: 0.0301 - accuracy: 0.9925 - val_loss: 0.5767 - val_accuracy: 0.6562
Epoch 5/15
23/23 [==============================] - 32s 1s/step - loss: 0.0271 - accuracy: 0.9924 - val_loss: 0.5896 - val_accuracy: 0.6587
Epoch 6/15
23/23 [==============================] - 29s 1s/step - loss: 0.0259 - accuracy: 0.9925 - val_loss: 0.5269 - val_accuracy: 0.6737
Epoch 7/15
23/23 [==============================] - 24s 1s/step - loss: 0.0251 - accuracy: 0.9923 - val_loss: 0.5165 - val_accuracy: 0.6759
Epoch 8/15
23/23 [==

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 32s 19ms/step - loss: 0.3485 - accuracy: 0.8267
Epoch 1/15
23/23 [==============================] - 26s 1s/step - loss: 0.0132 - accuracy: 0.9961 - val_loss: 0.5365 - val_accuracy: 0.7962
Epoch 2/15
23/23 [==============================] - 28s 1s/step - loss: 0.0123 - accuracy: 0.9966 - val_loss: 0.3490 - val_accuracy: 0.8561
Epoch 3/15
23/23 [==============================] - 25s 1s/step - loss: 0.0126 - accuracy: 0.9960 - val_loss: 0.4648 - val_accuracy: 0.8142
Epoch 4/15
23/23 [==============================] - 28s 1s/step - loss: 0.0121 - accuracy: 0.9960 - val_loss: 0.6347 - val_accuracy: 0.7753
Epoch 5/15
23/23 [==============================] - 25s 1s/step - loss: 0.0117 - accuracy: 0.9965 - val_loss: 0.3784 - val_accuracy: 0.8490
Epoch 6/15
23/23 [==============================] - 27s 1s/step - loss: 0.0113 - accuracy: 0.9966 - val_loss: 0.4744 - val_accuracy: 0.8154
Epoch 7/15
23/23 [==============================] - 25s 1s/step - l

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 37s 22ms/step - loss: 1.3058 - accuracy: 0.66111s - loss:
Epoch 1/15
23/23 [==============================] - 31s 1s/step - loss: 0.0085 - accuracy: 0.9973 - val_loss: 0.6100 - val_accuracy: 0.8012
Epoch 2/15
23/23 [==============================] - 27s 1s/step - loss: 0.0080 - accuracy: 0.9974 - val_loss: 0.4706 - val_accuracy: 0.8357
Epoch 3/15
23/23 [==============================] - 25s 1s/step - loss: 0.0078 - accuracy: 0.9976 - val_loss: 0.4994 - val_accuracy: 0.8320
Epoch 4/15
23/23 [==============================] - 22s 958ms/step - loss: 0.0076 - accuracy: 0.9978 - val_loss: 0.6390 - val_accuracy: 0.8047
Epoch 5/15
23/23 [==============================] - 25s 1s/step - loss: 0.0073 - accuracy: 0.9980 - val_loss: 0.5264 - val_accuracy: 0.8227
Epoch 6/15
23/23 [==============================] - 24s 1s/step - loss: 0.0074 - accuracy: 0.9977 - val_loss: 0.5376 - val_accuracy: 0.8204
Epoch 7/15
23/23 [==============================] - 24

...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 34s 20ms/step - loss: 2.9671 - accuracy: 0.6025
Epoch 1/15
23/23 [==============================] - 27s 1s/step - loss: 0.0049 - accuracy: 0.9984 - val_loss: 0.8096 - val_accuracy: 0.7934
Epoch 2/15
23/23 [==============================] - 22s 970ms/step - loss: 0.0043 - accuracy: 0.9985 - val_loss: 0.8772 - val_accuracy: 0.7849
Epoch 3/15
23/23 [==============================] - 21s 892ms/step - loss: 0.0042 - accuracy: 0.9986 - val_loss: 0.7781 - val_accuracy: 0.8027
Epoch 4/15
23/23 [==============================] - 25s 1s/step - loss: 0.0045 - accuracy: 0.9986 - val_loss: 0.6900 - val_accuracy: 0.8199
Epoch 5/15
23/23 [==============================] - 24s 1s/step - loss: 0.0042 - accuracy: 0.9986 - val_loss: 0.8447 - val_accuracy: 0.7951
Epoch 6/15
23/23 [==============================] - 22s 958ms/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 0.6959 - val_accuracy: 0.8182
Epoch 7/15
23/23 [==============================] - 25s 1s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 32s 18ms/step - loss: 3.1606 - accuracy: 0.60431s - loss: 3.3562 - accuracy - ETA: 0s - loss: 3.1835 - accuracy
Epoch 1/15
23/23 [==============================] - 29s 1s/step - loss: 0.0045 - accuracy: 0.9985 - val_loss: 0.5690 - val_accuracy: 0.8524
Epoch 2/15
23/23 [==============================] - 22s 968ms/step - loss: 0.0042 - accuracy: 0.9986 - val_loss: 0.8007 - val_accuracy: 0.8071
Epoch 3/15
23/23 [==============================] - 23s 995ms/step - loss: 0.0041 - accuracy: 0.9985 - val_loss: 0.7404 - val_accuracy: 0.8140
Epoch 4/15
23/23 [==============================] - 23s 1s/step - loss: 0.0035 - accuracy: 0.9989 - val_loss: 0.7491 - val_accuracy: 0.8117
Epoch 5/15
23/23 [==============================] - 24s 1s/step - loss: 0.0033 - accuracy: 0.9989 - val_loss: 0.7498 - val_accuracy: 0.8129
Epoch 6/15
23/23 [==============================] - 25s 1s/step - loss: 0.0034 - accuracy: 0.9988 - val_loss: 0.8995 - val_accuracy: 0.78

KeyboardInterrupt: 

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for Test Basic:  1.976881980895996  Loss for Test Plus:  9.3706693649292
Accuracy for Test Basic:  0.9105290770530701  Accuracy for Test Plus:  0.548263430595398
F1 for Test Basic:  0.8740143706811211  F1 for Test Plus:  0.5187715709466334
Precision for Test Basic:  0.8423697279453707  Precision for Test Plus:  0.4922905463396894
Recall for Test Basic:  0.9105290835256654  Recall for Test Plus:  0.5482634600014532


#### Dataset 7B 

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part5.csv')
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part6.csv')
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7B-Part7.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3, training4, training5, training6, training7]
num_nodes = 7
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1101.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5: 
            x, y = x6, y6
        else: 
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1101.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### Dataset 7C

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part5.csv')
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part6.csv')
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7C-Part7.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3, training4, training5, training6, training7]
num_nodes = 7
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1102.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5: 
            x, y = x6, y6
        else: 
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1102.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

### Experimenting with epochs and iterations 

Use of dataset 7A, considering it contains an equal partition of the data between the 7 nodes. 

#### #Epochs = 10, #Iterations = 10

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part5.csv')
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part6.csv')
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part7.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3, training4, training5, training6, training7]
num_nodes = 7
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1110.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5: 
            x, y = x6, y6
        else: 
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1110.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### #Epochs = 5, #Iterations = 5

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part5.csv')
training6 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part6.csv')
training7 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-7A-Part7.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3, training4, training5, training6, training7]
num_nodes = 7
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1111.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
x6, y6 = preprocessing(training6)
x7, y7 = preprocessing(training7)

xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2: 
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        elif node == 5: 
            x, y = x6, y6
        else: 
            x, y = x7, y7
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1111.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)